# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
weather_file="../output_data/cities.csv"
weather_df=pd.read_csv(weather_file)
weather_df


KeyError: 'Unnamed: 0'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
temp=list(weather_df["Max Temp"])
humidity=list(weather_df["Humidity"])
cloudiness=list(weather_df["Cloudiness"])
wind=list(weather_df["Wind Speed"])
#access gmaps with key
gmaps.configure(api_key=gkey)
ig=gmaps.figure(map_type="HYBRID")
heatmap_layer=gmaps.heatmap_layer(weather_df[["Lat","Lng"]], 
                               weights=weather_df["Humidity"], 
                               max_intensity=76, 
                               point_radius=20)


heatmap_layer.max_intensity=100
heatmap_layer.point_radius=10

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#the nulls values were all drop in weatherPy before saving the datafram in the csv

min_temp=int(input("Enter the Minimum Temperature in Fahrenheit between 0 and 95: "))
max_temp=int(input("Enter the Maximum Temperature in Fahrenheit between 0 and 95: "))
min_humidity=float(input("Enter the Minimum Humidity percentage between 0 and 100: "))
max_humidity=float(input("Enter the Maximum Humidity percentage between 0 and 100: "))
min_cloudiness=float(input("Enter the Minimum Cloudiness percentage between 0 and 100: "))
max_cloudiness=float(input("Enter the Maximum Cloudiness percentage between 0 and 100: "))
min_wind=float(input("Enter the Minimum Wind Speed (mph) between 0 and 100: "))
max_wind=float(input("Enter the Maximum Wind Speed (mph) between 0 and 100: "))

weather_conditions_df=weather_df[(weather_df["Max Temp"]>min_temp)&(weather_df["Max Temp"]<max_temp)&
                                       (weather_df["Humidity"]>min_humidity)&(weather_df["Humidity"]<max_humidity)&
                                       (weather_df["Cloudiness"]>min_cloudiness)&(weather_df["Cloudiness"]<max_cloudiness)&
                                       (weather_df["Wind Speed"]>min_wind)&(weather_df["Wind Speed"]<max_wind)]
weather_conditions_df                                   

Enter the Minimum Temperature in Fahrenheit between 0 and 95: 30
Enter the Maximum Temperature in Fahrenheit between 0 and 95: 75
Enter the Minimum Humidity percentage between 0 and 100: 15
Enter the Maximum Humidity percentage between 0 and 100: 55
Enter the Minimum Cloudiness percentage between 0 and 100: 30
Enter the Maximum Cloudiness percentage between 0 and 100: 50
Enter the Minimum Wind Speed (mph) between 0 and 100: 5
Enter the Maximum Wind Speed (mph) between 0 and 100: 15


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
239,239,manzhouli,35,CN,1558378819,45,49.60,117.43,45.24,13.67
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50
390,390,la ronge,40,CA,1558378869,33,55.10,-105.30,62.60,10.29
436,436,zaysan,43,KZ,1558378879,47,47.47,84.87,59.46,10.83
437,437,sioux lookout,40,CA,1558378879,44,50.10,-91.92,59.00,6.93
509,509,pilot butte,40,CA,1558378892,33,50.48,-104.42,59.00,10.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df=weather_conditions_df.copy()
hotel_df["Hotel Name"]=""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
239,239,manzhouli,35,CN,1558378819,45,49.60,117.43,45.24,13.67,
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50,
390,390,la ronge,40,CA,1558378869,33,55.10,-105.30,62.60,10.29,
436,436,zaysan,43,KZ,1558378879,47,47.47,84.87,59.46,10.83,
437,437,sioux lookout,40,CA,1558378879,44,50.10,-91.92,59.00,6.93,
509,509,pilot butte,40,CA,1558378892,33,50.48,-104.42,59.00,10.29,


In [42]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"
params={"key":gkey,"radius":"5000", "keyword":"Hotel"}

for index, row in hotel_df.iterrows():
    latitude=row["Lat"]
    longitude=row["Lng"]
    coordinates=f"{latitude}, {longitude}"
    params["location"]=coordinates

    data=requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=data["results"][0]["name"]
        city=row["City"]
    except:
        print(f"No hotels found in {city}")
        
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
239,239,manzhouli,35,CN,1558378819,45,49.60,117.43,45.24,13.67,Yenisai Hotel
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50,خوابگاه دانشجویی خاش
390,390,la ronge,40,CA,1558378869,33,55.10,-105.30,62.60,10.29,La Ronge Hotel & Suites
436,436,zaysan,43,KZ,1558378879,47,47.47,84.87,59.46,10.83,Rahat Hotel
437,437,sioux lookout,40,CA,1558378879,44,50.10,-91.92,59.00,6.93,Sunset Inn & Suites
509,509,pilot butte,40,CA,1558378892,33,50.48,-104.42,59.00,10.29,Sleep Inn Regina East


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))